In [2]:
con = mdb.connect(host="localhost", user=config['db_user'], db=config['db_name'], passwd=config['db_pass'])
database = MySQLDatabase(config['db_name'], user=config['db_user'], password=config['db_pass'])

In [6]:
fillPilotData()

Loading caption train2014 data...
SELECT * FROM image WHERE imageSubType = 'train2014'
0
0
Captions table for train2014 created.


In [12]:
len(new_list)

82783

In [10]:
split = 'train2014' # TODO
print 'Loading caption ' + split + ' data...'

cocoPath = '/home/hudaalamri/visdial-amt-chat/nodejs/static/train2014/' # TODO
captionsPath = '/home/hudaalamri/visdial-amt-chat/nodejs/static/annotations/' # TODO

f1 = open(os.path.join(captionsPath, 'captions_' + split + '.json'))
captionData = json.loads(f1.read())
f1.close()
imdir='COCO_%s_%012d.jpg'
subtype = split

image_path = cocoPath + subtype + '/'
print(image_path)
image_list = glob.glob(image_path + '*.jpg')

query = "SELECT * FROM image WHERE imageSubType = '" + split + "'"
print(query)

with con:
    cur = con.cursor()
    cur.execute(query)

    c = 0
    for i in range(cur.rowcount):
        row = cur.fetchone()
        if (image_path + row[1]) in image_list:
            image_list.remove(image_path + row[1])
            c +=1

    print c

count = 0
new_list = []
for name in image_list:
    new_list.append(name[len(image_path):])
    count = count +1

print len(new_list)

caption = defaultdict(list)
for cap in captionData['annotations']:
    image_name = imdir%(subtype, cap['image_id'])
    caption[image_name].append(cap)

imageData = []
for imname in new_list:
    imgid = caption[imname][0]['image_id']
    imageData.append({'imageId':str(imgid), 'imageName':imname, 'imageType':'mscoco', 'imageSubType':subtype, 'numHitsFinished':'0'})

with database.atomic():
    for idx in range(0, len(imageData), 200):
        print(idx)
        s = idx
        e = min(idx+200, len(imageData))
        Image.insert_many(imageData[s:e]).execute()

c = 0
captionData = []
for imname in new_list:
    c = c+1
    imgid = caption[imname][0]['image_id']
    idx = random.randint(0, len(caption[imname])-1)
    cap = caption[imname][idx] # random select one caption from the database.
    image = Image.get(Image.imageId == str(imgid))
    captionData.append({'captionId': cap['id'], 'caption': cap['caption'], 'image': image})

with database.atomic():
    for idx in range(0, len(captionData), 200):
        print(idx)
        s = idx
        e = min(idx+200, len(captionData))
        Caption.insert_many(captionData[s:e]).execute()

print "Captions table for " + split + " created."


Loading caption train2014 data...
/home/hudaalamri/visdial-amt-chat/nodejs/static/train2014/train2014/
SELECT * FROM image WHERE imageSubType = 'train2014'
0
82783
0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000
25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400
27600
27800
28000
28200
28400
28600
28800
29000
29200
29400
29600

In [1]:
import MySQLdb as mdb
import datetime
import sys
import os
from peewee import *
import glob
from collections import defaultdict
import json
import shortuuid
import random
from datetime import datetime

import PIL
from PIL import Image
from PIL import ImageFilter
from os import listdir
from os.path import isfile, join
from shutil import copyfile
import numpy

config = json.load(open('config.json', 'r'))

con = mdb.connect(host="localhost", user=config['db_user'], db=config['db_name'], passwd=config['db_pass'])
database = MySQLDatabase(config['db_name'], user=config['db_user'], password=config['db_pass'])
class BaseModel(Model):
    class Meta:
        database=database

class Image(BaseModel):
    imageId = CharField(primary_key=True, index=True)
    imageName = CharField()
    imageType = CharField()
    imageSubType = CharField()
    numHitsFinished = CharField()

class Caption(BaseModel):
    captionId = CharField(primary_key=True, index=True)
    caption = CharField()
    image = ForeignKeyField(Image)

class AMTHits(BaseModel):
    id = CharField(primary_key=True)
    socketId = CharField()
    assignmentId = CharField()
    workerId = CharField()
    approve = CharField(default='notApprove')
    hitId = CharField()
    status = CharField()
    isPaid = BooleanField(default='0')
    bonus = IntegerField(default=0)
    hitIden = CharField()
    comment = CharField()
    image = ForeignKeyField(Image)
    caption = ForeignKeyField(Caption)
    created_at = IntegerField(default=int(datetime.now().strftime('%s')))
    completed_at = IntegerField(default=0)

class Feedback(BaseModel):
    workerId = CharField()
    hitId = CharField()
    assignmentId = CharField()
    sequenceId = CharField()
    feedback = TextField()

class Question(BaseModel):
    id = CharField(primary_key=True, index=True)
    question = CharField()
    image = ForeignKeyField(Image)
    annotationId = ForeignKeyField(AMTHits)
    sequenceId = CharField()
    socketId = CharField()
    sourceId = CharField()
    destId = CharField()
    created_at = IntegerField(default=int(datetime.now().strftime('%s')))

class Answer(BaseModel):
    id = CharField(primary_key=True, index=True)
    answer = CharField()
    question = ForeignKeyField(Question)
    image = ForeignKeyField(Image)
    annotationId = ForeignKeyField(AMTHits)
    sequenceId = CharField()
    socketId = CharField()
    sourceId = CharField()
    destId = CharField()
    created_at = IntegerField(default=int(datetime.now().strftime('%s')))

def createDatabaseTables():
    database.connect()

    if not Image.table_exists():
        database.create_table(Image)
    if not Caption.table_exists():
        database.create_table(Caption)
    if not AMTHits.table_exists():
        database.create_table(AMTHits)
    if not Feedback.table_exists():
        database.create_table(Feedback)
    if not Question.table_exists():
        database.create_table(Question)
    if not Answer.table_exists():
        database.create_table(Answer)
    print "All database tables created."

def fillPilotData():

    split = 'train2014' # TODO
    print 'Loading caption ' + split + ' data...'

    cocoPath = '/home/hudaalamri/visdial-amt-chat/nodejs/static/dataset/' # TODO
    captionsPath = '/home/hudaalamri/visdial-amt-chat/nodejs/static/annotations/' # TODO

    f1 = open(os.path.join(captionsPath, 'captions_' + split + '.json'))
    captionData = json.loads(f1.read())
    f1.close()
    imdir='COCO_%s_%012d.jpg'
    subtype = split

    image_path = cocoPath + subtype + '/'
    image_list = glob.glob(image_path + '*.jpg')

    query = "SELECT * FROM image WHERE imageSubType = '" + split + "'"
    print(query)

    with con:
        cur = con.cursor()
        cur.execute(query)

        c = 0
        for i in range(cur.rowcount):
            row = cur.fetchone()
            if (image_path + row[1]) in image_list:
                image_list.remove(image_path + row[1])
                c +=1

        print c

    count = 0
    new_list = []
    for name in image_list:
        new_list.append(name[len(image_path):])
        count = count +1

    print len(new_list)

    caption = defaultdict(list)
    for cap in captionData['annotations']:
        image_name = imdir%(subtype, cap['image_id'])
        caption[image_name].append(cap)

    imageData = []
    for imname in new_list:
        imgid = caption[imname][0]['image_id']
        imageData.append({'imageId':str(imgid), 'imageName':imname, 'imageType':'mscoco', 'imageSubType':subtype, 'numHitsFinished':'0'})

    with database.atomic():
        for idx in range(0, len(imageData), 200):
            print(idx)
            s = idx
            e = min(idx+200, len(imageData))
            Image.insert_many(imageData[s:e]).execute()

    c = 0
    captionData = []
    for imname in new_list:
        c = c+1
        imgid = caption[imname][0]['image_id']
        idx = random.randint(0, len(caption[imname])-1)
        cap = caption[imname][idx] # random select one caption from the database.
        image = Image.get(Image.imageId == str(imgid))
        captionData.append({'captionId': cap['id'], 'caption': cap['caption'], 'image': image})

    with database.atomic():
        for idx in range(0, len(captionData), 200):
            print(idx)
            s = idx
            e = min(idx+200, len(captionData))
            Caption.insert_many(captionData[s:e]).execute()

    print "Captions table for " + split + " created."

def createRedisQueue():
    print "createRedisQueue called."

    import redis
    r = redis.StrictRedis(host='localhost', port=6379, db=0) # TODO

    with con:
        cur = con.cursor()
        cur.execute("SELECT * FROM image WHERE imageSubType = 'train2014' AND numHitsFinished = 0")
        r.delete('visdial_queue')
        count = 0
        for i in range(cur.rowcount):
            row = cur.fetchone()
            if(count < 5000): # push 5k images into queue
                print row[1], row[4]
                r.rpush('visdial_queue', row[1])
                count += 1

        print count

## ------------------------------------------------------------------------------------------------------
